In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
import hopsworks

project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME, api_key_value=config.HOPSWORKS_API_KEY
)

feature_store = project.get_feature_store()
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION
)

2025-03-05 16:30:08,565 INFO: Initializing external client
2025-03-05 16:30:08,566 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 16:30:16,070 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215683


In [6]:
# Create a feature view if it doesn't already exist
try:
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all(),
    )
    print(f"Feature view '{config.FEATURE_VIEW_NAME}' (version {config.FEATURE_VIEW_VERSION}) created successfully.")
except Exception as e:
    print(f"Error creating feature view: {e}")

# Retrieve the feature view
try:
    feature_view = feature_store.get_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
    )
    print(f"Feature view '{config.FEATURE_VIEW_NAME}' (version {config.FEATURE_VIEW_VERSION}) retrieved successfully.")
except Exception as e:
    print(f"Error retrieving feature view: {e}")

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1215683/fs/1203307/fv/time_series_hourly_feature_view/version/1
Feature view 'time_series_hourly_feature_view' (version 1) created successfully.
Feature view 'time_series_hourly_feature_view' (version 1) retrieved successfully.


In [ ]:
ts_data, _ = feature_view.training_data(
    description="Time-series hourly taxi rides"
)

In [ ]:
ts_data = ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

In [ ]:
ts_data.head()

In [ ]:
ts_data.info()

In [ ]:
ts_data["pickup_hour"].min()

In [ ]:
ts_data["pickup_hour"].max()

In [ ]:
import pandas as pd
ts_data["pickup_hour"] = pd.to_datetime(ts_data["pickup_hour"], errors="coerce")

In [ ]:
ts_data.info()

In [ ]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)  # Remove timezone

In [ ]:
ts_data.info()

In [ ]:
ts_data["year_month"] = ts_data["pickup_hour"].dt.to_period("M")  # Year-Month format

In [ ]:
ts_data

In [ ]:
import matplotlib.pyplot as plt

# Group by year_month and count
hour_counts = ts_data.groupby("year_month").size()

# Plot the data
ax = hour_counts.plot(kind="bar", figsize=(10, 6), color="skyblue", edgecolor="black")  # Use 'ax' to store the plot object
ax.set_title("Number of Hours by Year/Month", fontsize=16)
ax.set_xlabel("Year-Month", fontsize=12)
ax.set_ylabel("Count of Hours", fontsize=12)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
ax.grid(axis="y", linestyle="--", alpha=0.7)

plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()

In [ ]:
ts_data.info()

In [ ]:
gte = ts_data["year_month"] >= pd.Period("2023-01", freq="M")
lte = ts_data["year_month"] <= pd.Period("2023-12", freq="M")
cond = gte & lte
filtered_data = ts_data[cond].reset_index(drop=True)

In [ ]:
filtered_data.drop(columns=["year_month"], inplace=True) 

In [ ]:
filtered_data.shape

In [ ]:
ts_data = filtered_data

In [ ]:
from src.data_utils import transform_ts_data_info_features_and_target

features, targets = transform_ts_data_info_features_and_target(ts_data, window_size=24*28, step_size=23)

In [ ]:
features.sort_values(["pickup_location_id", "pickup_hour"])

In [ ]:
features_copy = features.copy()

In [ ]:
features_targets = features.copy()
features_targets["target"] = targets

features_targets.shape

In [ ]:
import pandas as pd
from src.data_utils import split_time_series_data

# Define the start and end dates
start_date = pd.Timestamp("2023-01-01")
end_date = pd.Timestamp("2023-12-31")

# Calculate the total number of days in the date range
total_days = (end_date - start_date).days + 1

# Calculate the cutoff date for 80% training data
cutoff_days = int(total_days * 0.8)
cutoff_date = start_date + pd.Timedelta(days=cutoff_days)

cutoff_date


In [ ]:
X_train, y_train, X_test, y_test = split_time_series_data(
    features_targets,
    cutoff_date=cutoff_date,
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
X_train.info()

In [ ]:
# Define the hyperparameter grid
param_grid = {
    'lgbmregressor__n_estimators': [100, 200, 300],
    'lgbmregressor__learning_rate': [0.01, 0.05, 0.1],
    'lgbmregressor__max_depth': [3, 5, 7],
    'lgbmregressor__colsample_bytree': [0.8, 0.9, 1.0],
    'lgbmregressor__subsample': [0.8, 0.9, 1.0],
    'lgbmregressor__reg_alpha': [0, 0.1, 0.5],
    'lgbmregressor__reg_lambda': [0, 0.1, 0.5]
}

In [ ]:
from src.pipeline_utils import get_pipeline
pipeline = get_pipeline()

In [ ]:
# Create RandomizedSearchCV object
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
import joblib
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_grid,
    n_iter=10,  # Number of parameter settings sampled
    scoring='neg_mean_absolute_error',  # Use negative MAE for scoring (as GridSearchCV maximizes the score)
    cv=3,  # Cross-validation folds
    verbose=1,
    random_state=42,
    n_jobs=-1  # Use all available cores
)

# Fit the RandomizedSearchCV object to the training data
random_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best parameters found:", random_search.best_params_)
print("Best negative MAE score:", random_search.best_score_)

# Get the best model
best_model = random_search.best_estimator_

# --- 7. Evaluate the Best Model ---
predictions = best_model.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f"Test MAE with the best model: {test_mae:.4f}")

# --- 8. Model Registry and Saving ---
# Save the best model
joblib.dump(best_model, config.MODELS_DIR / "lgb_model.pkl")

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(features)
output_schema = Schema(targets)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [ ]:
model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name="taxi_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor",
    input_example=features.sample(),
    model_schema=model_schema,
      
)
# https://community.hopsworks.ai/t/attributeerror-windowspath-object-has-no-attribute-startswith/1003
# model.save(config.MODELS_DIR / 'lgb_model.pkl')
model.save("D:/Taxi_Project/nyc_taxi/models/lgb_model.pkl")

In [ ]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

In [ ]:
from src.inference import get_model_predictions

In [ ]:
preds = get_model_predictions(model, features_copy)

In [ ]:
preds

In [ ]:
test_mae = mean_absolute_error(targets, preds["predicted_demand"])
print(f"{test_mae:.4f}")